# Computing expectation values of observables

A part from sampling the circuit in a basis, or extracting the state-vector in the ideal case, MPQP also allows to compute expectation values of observables with respect to the state generated by the circuit.

We first create a circuit for generating a 2-qubit state.

In [1]:
from mpqp import QCircuit
from mpqp.gates import *
circuit = QCircuit([H(0), Y(1), CNOT(0,1), Ry(2.6, 0), Ry(-0.87, 1)])
print(circuit)

     ┌───┐      ┌─────────┐ 
q_0: ┤ H ├──■───┤ Ry(2.6) ├─
     ├───┤┌─┴─┐┌┴─────────┴┐
q_1: ┤ Y ├┤ X ├┤ Ry(-0.87) ├
     └───┘└───┘└───────────┘


We then import the two needed objects: ``Observable`` and ``ExpectationMeasure``. The first takes as parameter a hermitian matrix (as a numpy array/matrix), and the second is the measure to be added to the circuit.

In [2]:
from mpqp.measures import Observable, ExpectationMeasure

In [3]:
import numpy as np
from mpqp.tools.maths import is_hermitian

matrix = np.array([[4,  2,  3, 8],
                   [2, -3,  1, 0],
                   [3,  1, -1, 5],
                   [8,  0,  5, 2]])
is_hermitian(matrix)

True

In [4]:
obs = Observable(matrix)

The ``ExpectationMeasure`` takes as parameter the list of qubits corresponding to the observable. The indices of qubits can be given non-ordered, non-contigous and restricted to some qubits, and MPQP will automatically operate on the circuit and the observable to handle that. One has also to precise the number of shots, when sampling the circuit to compute the expectation. If the number of shots is zero, the exact value is returned.

In [5]:
circuit.add(ExpectationMeasure([0,1], observable=obs, shots=0))

In [6]:
from mpqp.execution import run, ATOSDevice, IBMDevice
result = run(circuit, ATOSDevice.MYQLM_PYLINALG)
print(result)
print(result.expectation_value)

Result: ATOSDevice, MYQLM_PYLINALG
Expectation value: -3.3170640012883936
Error/Variance: 0.0


-3.3170640012883936


In [7]:
circuit = circuit.without_measurements()
circuit.add(ExpectationMeasure([0,1], observable=obs, shots=2000))

In [8]:
results = run(circuit, [ATOSDevice.MYQLM_PYLINALG, IBMDevice.AER_SIMULATOR])
print(results)

BatchResult: 2 results
Result: ATOSDevice, MYQLM_PYLINALG
Expectation value: -3.4069999999999996
Error/Variance: (0.143793296109391+0j)

Result: IBMDevice, AER_SIMULATOR
Expectation value: -3.2085
Error/Variance: 41.37162925


